In [55]:
import numpy as np
import pandas as pd
import requests
import json
import os

pd.set_option('display.max_columns', 100)




In [56]:
ASSET_ID = "276a8657-e718-4bef-a997-dd8daee9fcc4"

base_url = "http://0.0.0.0:5000/"

In [150]:
# Method to make API request to retrieve prices for a given asset ID between two specified Unix timestamps
def api_request_get_prices_between_unix_time(base_url, asset_id, unix_time_start, unix_time_end):

    # try:
    # Make GET request to API endpoint with query parameters for start and end Unix timestamps
    response = requests.get(base_url + "assets/" + asset_id + "/indicators_unix_between/",
                            params={'unix_time_start': unix_time_start, 'unix_time_end': unix_time_end})

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        print("get prices between status code: "+str(response.status_code))
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)

        # drop columns that are not needed
        df.drop(columns=['updated_at', 'created_at', 'date_time_gmt_5','ignore','asset_id'], inplace=True)

        # Return the DataFrame containing the retrieved price data
        return df

    else:
        print("Failed api_request_get_prices_between_unix_time")
        json_data = json.loads(response.content)

        print(json_data)
        return json_data
    


# Method to make API request to retrieve prices for a given asset ID between two specified Unix timestamps
def api_request_get_all_indicators_from_price(base_url, price_id,):

    # try:
    # Make GET request to API endpoint with query parameters for start and end Unix timestamps
    response = requests.get(base_url + "prices/" + price_id + "/indicators/")

    # Check if the request was successful
    if response.status_code == requests.codes.ok:
        print("get indicators status code: "+str(response.status_code))
        # Convert JSON response to Python dictionary
        json_data = json.loads(response.content)
        # Normalize the dictionary and convert it to a Pandas DataFrame
        df = pd.json_normalize(json_data)

        if df.empty:
            return df
        else:
        
            df.drop(columns=['id', 'created_at', 'updated_at','date_time_utc','unix_time','date_time_gmt_5'], inplace=True)

            df.set_index('price_id', inplace=True)

            pivoted_df = pd.pivot_table(df, index='price_id', columns='name', values='value', aggfunc=sum, fill_value=0)

            pivoted_df = pivoted_df.replace(0, np.nan)

            return pivoted_df

    else:
        print("Failed api_request_get_prices_between_unix_time")
        json_data = json.loads(response.content)

        print(json_data)
        return json_data
    

def get_indicators_and_merge(base_url, asset_id, unix_time_start, unix_time_end):

    df = api_request_get_prices_between_unix_time(base_url, asset_id, unix_time_start, unix_time_end)

    df_assets = pd.DataFrame()

    for i in df['id']:

        df2 = api_request_get_all_indicators_from_price(base_url, i)

        if not df2.empty:
            df_assets = pd.concat([df_assets, df2])
        
    if not df_assets.empty:
        df_merge = pd.merge(df, df_assets, left_on='id', right_on='price_id', how='left')
    else:
        df_merge = df

    return df_merge



In [151]:
df = get_indicators_and_merge(base_url, ASSET_ID, 1648749254, 1680982906)


get prices between status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get indicators status code: 200
get 

In [152]:
df


,high_price,unix_time,volume,date_time_utc,qav,num_trades,open_price,taker_base_vol,close_price,taker_quote_vol,low_price,id,accdist,accdit,accdit1,adx,adx_neg,adx_pos,ao,aroon_down,aroon_up,atr,bb_pb,bb_wb,bbh,bbhi,bbl,bbli,bbmavg,cci,cmf,dchb,dclb,dcmb,dcpb,dcwb,dpo,ema,emv,fi,ichimoku_a,ichimoku_b,ichimoku_base_line,ichimoku_conversion_line,kama,kchb,kchbi,kclb,kclbi,kcmb,...,kst,kst_sig,macd,macd_diff,macd_signal,mfi,mi,nvi,obv,ppo,psar_down_indicator,psar_up_indicator,pvo,roc,rsi,sma,smaemv,stc,stoch,stochrsi,trix,tsi,ulcer,uo,vi_neg,vi_pos,vpt,vwap,wma,wri,NaturalGas,Russell2000,cocoa,coffee,corn,cotton,dowJones,gold,nasdaq,oat,rice,silver,soybean,spy500,sugar,usBond,usBond10y,usBond2y,usBond5y,vwheat
0,46720.09,1648771200,56271.06474,2022-04-01T00:00:00+00:00,2.556533e+09,1372216,45510.35,28041.37621,46283.49,1.274360e+09,44200.00,eb208f87-c22d-461e-99ca-f0596f9ffa9f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,47213.00,1648857600,37073.53582,2022-04-02T00:00:00+00:00,1.721691e+09,1056716,46283.49,18562.93079,45811.00,8.621978e+08,45620.00,a2680f4b-86ca-4923-b026-65aa4561d30e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,47444.11,1648944000,33394.67794,2022-04-03T00:00:00+00:00,1.551000e+09,966563,45810.99,16740.18067,46407.35,7.776160e+08,45530.92,22e0b5c9-c952-476d-8080-93532424cc30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,46890.71,1649030400,44641.87514,2022-04-04T00:00:00+00:00,2.053435e+09,1171998,46407.36,21913.04154,46580.51,1.008164e+09,45118.00,ff593588-5a8f-477e-90e8-8342aa908c46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,47200.00,1649116800,42192.74852,2022-04-05T00:00:00+00:00,1.951668e+09,1046080,46580.50,20607.57735,45497.55,9.534947e+08,45353.81,299669b2-66c3-4fe8-b5db-93225681f57c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,28500.99,1680480000,79180.01405,2023-04-03T00:00:00+00:00,2.214361e+09,1651062,28171.87,40534.82008,27800.00,1.133866e+09,27200.24,9bb03fb6-941e-49d0-ac21-9e0745698e28,2.375458e+06,NaN,NaN,35.357609,16.273698,24.308454,2932.552882,4.0,88.0,12